In [ ]:
import glob
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
DIR_ITERATIVE_BASELINE = (
    "../experiments/outputs/compositional_graphics/logo_stitch_iterative/*"
)
DIR_ITERATIVE_CODEX = (
    "../experiments/outputs/compositional_graphics/logo_codex_stitch_iterative/*"
)

In [ ]:
def get_log_likelihoods(dir_results):
    data = []

    for path in sorted(glob.glob(dir_results)):
        config_json_path = os.path.join(path, "config.json")
        with open(config_json_path, "r") as f:
            config = json.load(f)
        global_batch_size = config["experiment_iterator"]["task_batcher"]["params"][
            "global_batch_size"
        ]

        test_likelihoods_json_path = os.path.join(path, "0", "test_likelihoods.json")
        with open(test_likelihoods_json_path, "r") as f:
            likelihoods_data = json.load(f)

        stitch_frontiers_json_path = os.path.join(
            path, "0", "train", "stitch_frontiers.json"
        )
        with open(stitch_frontiers_json_path, "r") as f:
            stitch_frontiers_data = json.load(f)

        for task_name, ll_list in likelihoods_data["log_likelihoods_by_task"][
            "test"
        ].items():
            data.append(
                {
                    "batch_size": global_batch_size,
                    "task_name": task_name,
                    "log_likelihood": ll_list[0],
                    "n_frontiers": len(stitch_frontiers_data["frontiers"]),
                }
            )

    df = pd.DataFrame(data)
    return df

In [ ]:
df1 = get_log_likelihoods(DIR_ITERATIVE_BASELINE)
df1["condition"] = "Stitch"

df2 = None

df2 = get_log_likelihoods(DIR_ITERATIVE_CODEX)
df2["condition"] = "Stitch + Codex"

In [ ]:
df = pd.concat([df1, df2], axis=0).reset_index(drop=True)

In [ ]:
g = sns.lineplot(data=df, x="batch_size", y="log_likelihood", hue="condition")
g.set(xscale="log");

In [ ]:
sns.pointplot(data=df, x="batch_size", y="log_likelihood", hue="condition")

In [ ]:
sns.pointplot(data=df, x="batch_size", y="n_frontiers", hue="condition")

In [ ]:
sns.pointplot(data=df, x="n_frontiers", y="log_likelihood", hue="condition")

## What programs does Codex generate?

In [ ]:
BATCH_SIZE = 25
codex_query_results_json_path = f"../experiments/outputs/compositional_graphics/logo_codex_stitch_iterative/logo_codex_stitch_iterative_{BATCH_SIZE}/0/codex_query_results.json"

with open(codex_query_results_json_path, "r") as f:
    codex_query_results = json.load(f)

In [ ]:
data = []
for p in codex_query_results["prompt_programs"]:
    data.append(
        {
            "program": p,
            "origin": "train",
            "valid": True,
        }
    )
for p in codex_query_results["programs_valid"]:
    data.append(
        {
            "program": p,
            "origin": "codex",
            "valid": True,
        }
    )
for p in codex_query_results["programs_invalid"]:
    data.append(
        {
            "program": p,
            "origin": "codex",
            "valid": False,
        }
    )

df = pd.DataFrame(data)
df["program_str_len"] = df.program.str.len()

In [ ]:
print(f"+ From {BATCH_SIZE} training programs, Codex generated {len(df[df.origin == 'codex'])} programs, for a total of {len(df)} programs.")
print(f"+ Of these, {len(df[(df.origin == 'codex') & (df.valid)])} were valid and {len(df[(df.origin == 'codex') & (~df.valid)])} were invalid.")
print(f"+ In total, there were {df['program'].nunique()} unique programs; {df[df.valid]['program'].nunique()} were valid.")

copied_programs = set(codex_query_results["programs_valid"]) & set(codex_query_results["prompt_programs"])
print(f"+ {len(copied_programs)} of the Codex programs were direct copies from the training data.")

In [ ]:
sns.displot(data=df, x="program_str_len", hue="valid", col="origin")

In [ ]:
df.program[df.origin == "train"].tolist()

In [ ]:
df.program[df.origin == "codex"].tolist()